In [ ]:
# Do two steps:
# Step 1: Merge landmark detection (from RTM Pose) with Person detector (ID):

# REASSIGNMENT === UPDATED SCRIPT: Add Full Skeleton ===
# === FINAL CLEAN SCRIPT: Stable ID Assignment + Pose Matching + True Reassignment (Skeleton to Skeleton) ===

# === IMPORTS ===
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm

# === SETTINGS ===
MATCH_THRESHOLD_FRAME = 150          # Pixel-based threshold if not normalizing
USE_NORMALIZED_DISTANCE = True       # Switch for normalized matching, otherwise fixed distance matching
NORMALIZED_MATCH_THRESHOLD = 0.3     # Threshold if normalizing distances

# === LOAD VIDEO FILES ===
base_folder = Path("/Users/Christian/Downloads/Microadaptive Teaching Dritter Teil - LAs/Marlon/Erste Sitzung/YOLO")
video_files = list(base_folder.glob("*.mp4")) + list(base_folder.glob("*.MP4"))

# === MAIN LOOP: PROCESS EACH VIDEO ===
for video_path in video_files:
    print(f"\n🎥 Processing video: {video_path.name}")

    INPUT_PATH = video_path

    # === LOAD TRACKING DATA (Bounding Boxes) ===
    tracking_df = pd.read_csv(INPUT_PATH.parent / "YOLO" / (INPUT_PATH.stem + ".csv"))

    # === LOAD POSE ESTIMATION DATA (Skeletons) ===
    pose_df = pd.read_csv(INPUT_PATH.parent / "DLC" / (INPUT_PATH.stem + "_predictions.csv"), low_memory=False)

    # === PREPARE POSE DATAFRAME ===
    multi_index = pd.MultiIndex.from_arrays(pose_df.iloc[0:3].values, names=["individual", "bodypart", "coord"])
    pose_data_cleaned = pose_df.iloc[3:].copy()
    pose_data_cleaned.columns = multi_index
    pose_data_cleaned.reset_index(drop=True, inplace=True)
    pose_data_cleaned["Frame"] = pose_data_cleaned.index.astype(int)

    # === DETECT BODY PARTS (skip non-body columns) ===
    bodyparts = pose_data_cleaned.columns.get_level_values("bodypart").unique()
    bodyparts = [bp for bp in bodyparts if bp not in ["Frame", "scorer"]]

    # === FINAL DATA COLLECTOR ===
    final_data = []

    # === FRAME-BY-FRAME PROCESSING ===
    for frame_idx, pose_row in tqdm(pose_data_cleaned.iterrows(), total=len(pose_data_cleaned), desc=f"Processing {INPUT_PATH.stem}"):
        frame_num = int(pose_row["Frame"].iloc[0] if isinstance(pose_row["Frame"], pd.Series) else pose_row["Frame"])
        frame_boxes = tracking_df[tracking_df.Frame == frame_num]

        # === EXTRACT ALL SKELETON CENTERS IN THIS FRAME ===
        skeleton_centers = {}
        individuals = pose_row.index.get_level_values("individual").unique()
        individuals = [ind for ind in individuals if ind.startswith("idv_")]

        for ind in individuals:
            keypoints = []
            for bp in bodyparts:
                try:
                    x = float(pose_row[(ind, bp, "x")])
                    y = float(pose_row[(ind, bp, "y")])
                    if pd.notna(x) and pd.notna(y):
                        keypoints.append((x, y))
                except:
                    continue
            if keypoints:
                cx, cy = np.mean(keypoints, axis=0)  # Compute mean x, mean y = skeleton center
                skeleton_centers[ind] = (cx, cy)

        # === MATCH EACH BOUNDING BOX TO NEAREST SKELETON ===
        for _, box in frame_boxes.iterrows():
            x_center = (box.X1 + box.X2) / 2
            y_center = (box.Y1 + box.Y2) / 2
            pid = int(box.Person_ID)

            best_match = None
            min_distance = float('inf')

            for skel_id, (cx, cy) in skeleton_centers.items():
                center_distance = np.linalg.norm(np.array([cx, cy]) - np.array([x_center, y_center]))

                # === OPTIONAL: Normalize distance by bounding box size ===
                if USE_NORMALIZED_DISTANCE:
                    bbox_width = box.X2 - box.X1
                    bbox_height = box.Y2 - box.Y1
                    avg_bbox_size = (bbox_width + bbox_height) / 2
                    if avg_bbox_size > 0:
                        center_distance /= avg_bbox_size

                threshold = NORMALIZED_MATCH_THRESHOLD if USE_NORMALIZED_DISTANCE else MATCH_THRESHOLD_FRAME

                # === KEEP CLOSEST SKELETON UNDER THRESHOLD ===
                if center_distance < min_distance and center_distance < threshold:
                    best_match = skel_id
                    min_distance = center_distance

            # === SAVE MATCH INFORMATION ===
            data = {
                "Frame": frame_num,
                "Old_ID": pid,
                "New_ID": pid,
                "X1": box.X1,
                "Y1": box.Y1,
                "X2": box.X2,
                "Y2": box.Y2,
            }

            if best_match:
                skel_num = best_match.split("_")[1]
                for bp in bodyparts:
                    x = pose_row.get((f"idv_{skel_num}", bp, "x"), np.nan)
                    y = pose_row.get((f"idv_{skel_num}", bp, "y"), np.nan)
                    conf = pose_row.get((f"idv_{skel_num}", bp, "likelihood"), np.nan)
                    data[f"{bp}_x"] = x
                    data[f"{bp}_y"] = y
                    data[f"{bp}_conf"] = conf

            final_data.append(data)

    # === SAVE FINAL CORRECTED CSV ===
    final_df = pd.DataFrame(final_data)
    final_df = final_df.sort_values(["Frame", "New_ID"])
    final_df.drop(columns=["bodyparts_x", "bodyparts_y", "bodyparts_conf", "_x", "_y", "_conf"], errors="ignore", inplace=True)

    OUTPUT_FOLDER = INPUT_PATH.parent / "corrected"
    OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

    output_csv_name = f"{INPUT_PATH.stem}_corrected.csv"
    final_df.to_csv(OUTPUT_FOLDER / output_csv_name, index=False)

    print("\u2705 Final corrected CSV saved for:", video_path.name)

print("\n🎉 All sessions processed!")

In [ ]:
# Step 2:  Merge landmark and ID detection with object detection: 

import pandas as pd
from pathlib import Path

# === PATHS ===
main_folder = Path("/Users/Christian/Downloads/Javelin training/analysis/corrected")
object_folder = Path("/Users/Christian/Downloads/Javelin training/analysis/YOLO_Object")
merged_folder = main_folder / "with_objects"
merged_folder.mkdir(parents=True, exist_ok=True)

# === PROCESSING LOOP ===
for corrected_file in main_folder.glob("*_corrected.csv"):
    base_name = corrected_file.stem.replace("_corrected", "")
    object_file = object_folder / f"{base_name}.csv"

    if not object_file.exists():
        print(f"Skipping {base_name} — no object file found.")
        continue

    # Load both dataframes
    df_people = pd.read_csv(corrected_file)
    df_object = pd.read_csv(object_file)

    # Compute center points
    df_object["center_x"] = (df_object["X1"] + df_object["X2"]) / 2
    df_object["center_y"] = (df_object["Y1"] + df_object["Y2"]) / 2

    # Pivot: one row per frame, columns like Tip_center_x, Handle_center_y, etc.
    # Keep only needed columns
    df_object_slim = df_object[["Frame", "Label", "X1", "Y1", "X2", "Y2", "center_x", "center_y"]]
    
    # Pivot: one row per frame, columns like Tip_X1, Tip_center_x, etc.
    df_object_pivot = df_object_slim.pivot_table(
        index="Frame",
        columns="Label",
        aggfunc="first"
    )
    

    # Flatten multi-index columns
    df_object_pivot.columns = [f"{label}_{coord}" for coord, label in df_object_pivot.columns]
    df_object_pivot.reset_index(inplace=True)

    # Merge person + object center info
    df_merged = df_people.merge(df_object_pivot, on="Frame", how="left")

    # Save to new folder
    out_path = merged_folder / f"{base_name}_merged.csv"
    df_merged.to_csv(out_path, index=False)
    print(f"✅ Merged and saved: {out_path.name}")

print("\n🎉 All sessions processed!")